# Generate Map

> Class that generates the data map

In [ ]:
#| default_exp app

In [ ]:
#| export
from nbdev.showdoc import *
# TODO work out how to get around below hack
try:
    from SolomonIslandsDataMap.load_data import *
    from SolomonIslandsDataMap.dash_components import *
except: 
    from load_data import *
    from dash_components import *
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio # Unless this is used graphs will not be dynamic?
import json
from git import Repo
import pandas as pd
import numpy as np
from fastcore.test import *
from dash import Dash, dcc, Output, Input, html, Patch, ctx  # pip install dash
import dash_bootstrap_components as dbc    # pip install dash-bootstrap-components
from dash_bootstrap_templates import load_figure_template
import random
import dash_mantine_components as dmc

In [ ]:
# TODO - do a need a state class that tracks the state of different website objects to help with
# updating etc. 
# TODO - not even state, should it be an object that contains everythin to track. Execpt if I am patching,
# then the tracking in mem is unessary, so better to just track necesary values like: last changes card etc. 

## Load Data

In [ ]:
#| export
sol_geo = SolomonGeo.load_pickle("/testData/")
geo_df = sol_geo.geo_df

In [ ]:
fig = define_map(sol_geo)

In [ ]:
fig['data'][0]['visible']


True

## Setup Dash Components

In [ ]:
#| export
# Build your components
# FYI the best themes seem to be: [Darkly, Flatly, Minty, Slate, JOURNAL]
app = Dash(__name__, external_stylesheets=[dbc.themes.MINTY])
server = app.server
load_figure_template("minty")

geos = geo_df.loc[:, 'agg'].unique()
cen_vars = sol_geo.census_vars
mytitle = dcc.Markdown(children="## " + cen_vars[-1] + " by " + geos[0]) # TODO This needs a default title
map_graph = dcc.Graph(figure=define_map(sol_geo))
# TODO entire accordian will need to be the child
cards = dbc.Accordion(children= 
        card_list(sol_geo, "Population - Total", sol_geo.geo_levels[0], sol_geo.census_vars[-1]),
        always_open=True,
        class_name = "accordion-header",
    )
               

#dropdown_geo = dbc.Dropdown(options=geos,
#                        value=geos[0],  # initial value displayed when page first loads
#                        clearable=False)
dropdown_geo = dmc.SegmentedControl(
                            id="segmented",
                            value=geos[0],
                            data=geos,
                             orientation="vertical",
                            color = 'gray',
                            fullWidth = True,
                            # TODO - think there is a version issue with class_name on the server, need to fix
                            #className="btn-group btn-primary",
                            #class_name = "btn btn-primary"
                            #color = dmc.theme.DEFAULT_COLORS["teal"][3]
    # TODO this color functionality is beyond stupid...
    # TODO definitely change to dbc, even though more complicated get consistent theme.s..
                        ) # TODO consider redoing as theme is not consistent with this library
'''
dropdown_geo = dbc.ButtonGroup(
    [
        dbc.RadioItems(
            id="radios",
            className="btn-group",
            inputClassName="btn-check",
            labelClassName="btn btn-outline-primary",
            labelCheckedClassName="active",
            options=[
                {"label": geos[0], "value": geos[0]},
                {"label": geos[1], "value": geos[1]},
                {"label": geos[2], "value": geos[2]},
            ],
            value=geos[0],
        ),
        #html.Div(id="output"),
    ],
    className="radio-group",
    vertical = True,
)'''
dropdown_var = dcc.Dropdown(options=cen_vars,
                        value=cen_vars[-1],  # initial value displayed when page first loads
                        clearable=False,)

navbar = dbc.NavbarSimple(
    children=[
        dbc.NavItem(dbc.NavLink("Census Data", href="#")),
        dbc.DropdownMenu(
            children=[
                dbc.DropdownMenuItem("More pages coming soon", header=True),
                dbc.DropdownMenuItem("Population Projection", href="#"),
            ],
            nav=True,
            in_navbar=True,
            label="More",
        ),
    ],
    brand="Solomon Islands Data Map",
    brand_href="#",
    color="primary",
    #dark=True,
    class_name="navbar navbar-expand-lg bg-primary"
)


## Create a sidebar
The sidebar will 

In [ ]:
#| export
# Note, for now I am not using a sidebar style as I do not want to fix the width
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "24rem",
    "padding": "2rem 1rem",
    #"background-color": "#f8f9fa",
}


sidebar = html.Div(
    [
        html.H2("Filters"),
        html.Hr(),
        dbc.Nav(
            [
                html.P("Geography"), # TODO add a tooltip button here with link to geo explanation
                dropdown_geo,
                html.Br(),
                html.P("Data"), # TODO add a little info button here with link to geo explanation
                dropdown_var,
                #html.Br(),
                #dcc.Dropdown(id = 'three')

            ],
            vertical=True,
            pills=True,
        ),
    ],
    #style=SIDEBAR_STYLE,
)


## Customize the Layout

In [ ]:
#| export
app.layout = dbc.Container([
                dbc.Row([
                    navbar
                ]),
                dbc.Row(
                    [dbc.Col(sidebar, width = 2),
                    dbc.Col([mytitle,
                             map_graph,
                            dbc.Row([cards])
                            ], width = 10)#, style = {'margin-left':'15px', 'margin-top':'7px', 'margin-right':'15px'})
                     ], justify = 'center'),                    
                ], fluid = True)

## Build callbacks to create interaction

In [ ]:
#| export
# Callback allows components to interact

# TODO put title in it's own callback
@app.callback(
    Output(map_graph, 'figure'),
    Output(mytitle, 'children'),
    Input(dropdown_geo, 'value'),
    Input(dropdown_var, 'value'),
    allow_duplicate=True,
    prevent_initial_call=True
)
def update_geography(geo_input:str, # User input from the geography dropdown
                     census_var:str # User input for the census variable
              )->(type(go.Figure()), str): # Returns a graph object figure after being updated and the dynamic title
    '''
    Updates the focus census variable or geography dispalayed on the map
    '''
    patched_figure = Patch()
    button_clicked = ctx.triggered_id
    print(dropdown_geo)
    if button_clicked == dropdown_geo.id:
        # Update disaplayed geography based on 
        for geo in sol_geo.geo_levels:
            i = np.where(sol_geo.geo_levels == geo)[0][0] # Tracks the trace number
            patched_figure['data'][i]['visible'] = geo_input == geo
            print(geo_input == geo)
    elif button_clicked == dropdown_var.id:
        # Update the z values in map to the data for the requested
        # census variable
        
        for geo in sol_geo.geo_levels:
        # Ar updates the z value ie. data disaplyed each time
        # TODO this is fairly inefficient, as we are processing each time
        # Maybe faster framework like polars could help? or caching but would require a lot of caching
            i = np.where(sol_geo.geo_levels == geo)[0][0] # Tracks the trace number
            ar = sol_geo.get_df(agg_filter = geo, var_filter = census_var).values
            ar = ar.reshape((ar.shape[0],))
            patched_figure['data'][i]['z'] = ar

    # returned objects are assigned to the component property of the Output
    return patched_figure, '## Solomon Islands Data map - ' + geo_input

On click, update the contents of the kpi's

In [ ]:
#| hide
show_doc(update_geography)

---

[source](https://github.com/Gippers/SolomonIslandsDataMap/blob/main/SolomonIslandsDataMap/app.py#L157){target="_blank" style="float:right; font-size:smaller"}

### update_geography

>      update_geography (geo_input:str, census_var:str)

Updates the focus census variable or geography dispalayed on the map

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| geo_input | str | User input from the geography dropdown |
| census_var | str | User input for the census variable |
| **Returns** | **(<class 'plotly.graph_objs._figure.Figure'>, <class 'str'>)** | **Returns a graph object figure after being updated and the dynamic title** |

In [ ]:
#| export
@app.callback(
    Output(cards, 'children'),
    # TODO - make this a Row object with children, then use function to recontruct
    # a group of them
    Input(map_graph, 'clickData'),
    prevent_initial_call=True
)
def update_kpis(clickData):
    # TODO - I need to know the current variable selection in order to make this selection correct.
    # TODO - I also need to reset this when the filter is changed
    # TODO - This callback should be triggered by the main callback https://dash.plotly.com/advanced-callbacks see callbacks as an indirect
    # result section
    if clickData is None:
        # TODO when none, maybe in future return current saved state, for now doing total
        # TODO add a heading and maybe put in an acordian
        new_cards = card_list(sol_geo, 
                              sol_geo.geo_levels[0], 
                              sol_geo.census_vars[-1])
        return new_cards
    else:
        print(clickData)
        location = clickData['points'][0]['location']
        # filter dataframe by store location, then sum all orders of that store.
        new_cards = card_list(sol_geo, 
                              "Population - " + location,
                              sol_geo.geo_levels[0], 
                              sol_geo.census_vars[-1], 
                              [location])
        
        # get all orders from this stor location 

        return new_cards

In [ ]:
fig['data'][2]['z'].shape

(10,)

In [ ]:
# TODO Do the title in one callback that listens for changes to either of them

In [ ]:
ar = sol_geo.get_df(agg_filter = 'province', var_filter = 'female_pop').values
ar = ar.reshape((ar.shape[0],))
ar.shape

(10,)

In [ ]:
#patched_figure = Patch()
user_input = 'constituency'
for geo in sol_geo.geo_levels:
    i = np.where(sol_geo.geo_levels == geo)[0][0] # Tracks the trace number
    fig['data'][i]['visible'] = user_input == geo
    print(user_input == geo)

False
True
False


In [ ]:
# TODO undo hardcoding
test_eq(fig['data'][0]['visible'], False)
test_eq(fig['data'][1]['visible'], True)
test_eq(fig['data'][2]['visible'], False)

### Update the colour columns based on data selection

In [ ]:
# TODO the above demonstrates that we can stack mutliple graphs and toggle the visibility and potentially z value in order to 
# quickly update our graph

fig = px.choropleth_mapbox(sol_geo.get_df(agg_filter = 'ward'),
        # TODO - may need to use old version of mapping to get visibility hack to work..
                        #geojson = str(repo.working_tree_dir) + "/assets/sol_geo.json",
                            geojson=sol_geo.get_geojson(agg_filter = 'ward'),
                           locations=sol_geo.get_df(agg_filter = 'ward').index,
                               color = 'total_pop',
                           color_continuous_scale="deep",
                               mapbox_style = 'carto-positron',
                               opacity = 0.8,
                        zoom = 5,
                        center={"lat": -9.565766, "lon": 162.012453})
print(fig["data"][0]["z"])

## Run the dash app

In [ ]:
#| export
#| server
# Run app
if __name__=='__main__':
    try:
        app.run_server(debug=True, port = random.randint(1000, 9999)) # Random int mitigates port collision
    except:
        print("Cannot run server here")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()